Adapted from http://www.cyber-omelette.com/2016/11/python-n-body-orbital-simulation.html

In [ ]:
import math
import time

import matplotlib.pyplot as plt
from astropy import constants as const

%matplotlib notebook

In [ ]:
class Point:
    def __init__(self, x,y,z):
        self.x = x
        self.y = y
        self.z = z


class Body:
    def __init__(self, location, mass, velocity, name=""):
        self.location = location
        self.mass = mass
        self.velocity = velocity
        self.name = name
        
    @classmethod
    def from_dict(cls, d, name=''):
        return cls(location=d["location"],
                   mass=d["mass"],
                   velocity=d["velocity"],
                   name=name)

In [ ]:
def calculate_single_body_acceleration(bodies, body_index):
    G_const = const.G.si.value
    acceleration = Point(0, 0, 0)
    target_body = bodies[body_index]
    for index, external_body in enumerate(bodies):
        if index != body_index:
            r = ((target_body.location.x - external_body.location.x)**2 +
                 (target_body.location.y - external_body.location.y)**2 +
                 (target_body.location.z - external_body.location.z)**2)
            r = math.sqrt(r)
            tmp = G_const * external_body.mass / r**3
            acceleration.x += tmp * (external_body.location.x - target_body.location.x)
            acceleration.y += tmp * (external_body.location.y - target_body.location.y)
            acceleration.z += tmp * (external_body.location.z - target_body.location.z)

    return acceleration

In [ ]:
def compute_velocity(bodies, time_step=1):
    for body_index, target_body in enumerate(bodies):
        acceleration = calculate_single_body_acceleration(bodies, body_index)
        target_body.velocity.x += acceleration.x * time_step
        target_body.velocity.y += acceleration.y * time_step
        target_body.velocity.z += acceleration.z * time_step 

In [ ]:
def update_location(bodies, time_step=1):
    for target_body in bodies:
        target_body.location.x += target_body.velocity.x * time_step
        target_body.location.y += target_body.velocity.y * time_step
        target_body.location.z += target_body.velocity.z * time_step

In [ ]:
def compute_gravity_step(bodies, time_step=1):
    compute_velocity(bodies, time_step=time_step)
    update_location(bodies, time_step=time_step)

In [ ]:
def run_simulation(bodies, names=None, time_step=1, number_of_steps=10000, report_freq=100):
    #create output container for each body
    body_locations_hist = []
    for current_body in bodies:
        body_locations_hist.append({"x":[], "y":[], "z":[], "name": current_body.name})

    for i in range(1,number_of_steps):
        compute_gravity_step(bodies, time_step=time_step)
        
        if i % report_freq == 0:
            for index, body_location in enumerate(body_locations_hist):
                body_location["x"].append(bodies[index].location.x)
                body_location["y"].append(bodies[index].location.y)      
                body_location["z"].append(bodies[index].location.z)

    return body_locations_hist   

In [ ]:
def plot_output(bodies):
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1, projection='3d')
    max_range = 0
    for i, current_body in enumerate(bodies):
        max_dim = max(max(current_body["x"]), max(current_body["y"]), max(current_body["z"]))
        if max_dim > max_range:
            max_range = max_dim
        ax.plot(current_body["x"], current_body["y"], current_body["z"],
                c=colours[i], label=current_body["name"])        

    ax.set_xlim([-max_range, max_range])    
    ax.set_ylim([-max_range, max_range])
    ax.set_zlim([-max_range, max_range])
    ax.legend()        
    
    plt.show()

In [ ]:
def plot_output_timelapse(motions):
    n_steps = len(motions[0]['x'])
    n_bodies = len(motions)
    
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1, projection='3d')
    
    for j in range(n_steps):
        for i, current_body in enumerate(motions):
            ax.scatter(current_body["x"][j], current_body["y"][j], current_body["z"][j], c=colours[i])
        fig.canvas.draw()
        time.sleep(0.5)
        ax.cla()

In [ ]:
# Planet data (location (m), mass (kg), velocity (m/s)
sun = {"location": Point(0,0,0), "mass": 2e30, "velocity": Point(0,0,0)}
mercury = {"location": Point(0,5.7e10,0), "mass": 3.285e23, "velocity": Point(47000,0,0)}
venus = {"location": Point(0,1.1e11,0), "mass": 4.8e24, "velocity": Point(35000,0,0)}
earth = {"location": Point(0,1.5e11,0), "mass": 6e24, "velocity": Point(30000,0,0)}
mars = {"location": Point(0,2.2e11,0), "mass": 2.4e24, "velocity": Point(24000,0,0)}
#jupiter = {"location": Point(0,7.7e11,0), "mass": 1e28, "velocity": Point(13000,0,0)}
jupiter = {"location": Point(0,7.7e11,0), "mass": 1e28, "velocity": Point(0,13000,0)}  # Fling it to the Sun!
saturn = {"location": Point(0,1.4e12,0), "mass": 5.7e26, "velocity": Point(9000,0,0)}
uranus = {"location": Point(0,2.8e12,0), "mass": 8.7e25, "velocity": Point(6835,0,0)}
neptune = {"location": Point(0,4.5e12,0), "mass": 1e26, "velocity": Point(5477,0,0)}
pluto = {"location": Point(0,3.7e12,0), "mass": 1.3e22, "velocity": Point(4748,0,0)}

In [ ]:
# Build list of planets in the simulation, or create your own
bodies = [
    Body.from_dict(sun, name='Sun'),
    Body.from_dict(mercury, name='Mercury'),
    Body.from_dict(venus, name='Venus'),
    Body.from_dict(earth, name='Earth'),
    Body.from_dict(mars, name='Mars'),
    Body.from_dict(jupiter, name='Jupiter'),
    Body.from_dict(saturn, name='Saturn'),
    Body.from_dict(uranus, name='Uranus'),
    Body.from_dict(neptune, name='Neptune'),
    Body.from_dict(pluto, name='Pluto')
]

colours = ['yellow', 'gray', 'salmon', 'green', 'red', 'brown', 'orange', 'aquamarine', 'skyblue', 'peachpuff']

In [ ]:
# Takes a while. This is why no one do serious n-body in Python.
motions = run_simulation(bodies, time_step=7200, number_of_steps=80000, report_freq=1000)

In [ ]:
plot_output_timelapse(motions)